In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from numpy import absolute
from numpy import sqrt
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
#pre processing data
df = pd.read_csv('DIM_GPU_PROD.csv')
ids = df['Id']
drops = ["Processor_Manufacturer", "GPU_Manufacturer", "Memory_Type","Id",df.columns[0], df.columns[1]]
df = df.drop(drops,axis =1)
newdf = pd.read_csv('FACT_CRYPTO_RATE.csv')
newdf = newdf[((newdf['CodeId']==6) |( newdf['CodeId'] == 2))&(newdf['TimeId'] >= 20140917)]
newdf = newdf[['Open','Close','TimeId']]
newdf = newdf.head(2054)
df["Close"] = newdf["Close"].values
df["Open"] = newdf["Open"].values
df["Processor"] = pd.factorize(df['Processor'])[0]
X = df.values[:, 0:]
prices_df = pd.read_csv("FACT_GPU_PRICE.csv")
y = prices_df["Price_USD"].head(2054).values


In [3]:
df

,Processor,Memory_Capacity,Close,Open
0,0,2.000,0.023456,0.023920
1,1,2.000,0.021769,0.023458
2,1,4.000,0.020249,0.021776
3,2,4.000,0.020972,0.020265
4,3,0.125,0.020455,0.020954
...,...,...,...,...
2049,287,16.000,0.014213,0.014948
2050,288,2.000,0.010971,0.014202
2051,289,24.000,0.012846,0.011051
2052,289,24.000,0.012644,0.012881


In [4]:
#split into training and test set
X_train,X_test, y_train,y_test = train_test_split(X,y, test_size = 0.2,random_state = 1)


#fit data
linreg = LinearRegression()


In [5]:
#PART A 

lin_error_arr = []
lin_value = [i for i in range(2,30)]
best_fold = 2
min_error = 30
for i in range(2,30):
    cv = KFold(n_splits=i, random_state=1, shuffle=True)
    scores = cross_val_score(linreg, X_train, y_train,scoring='neg_mean_squared_error',cv=cv)
    print(i,"-FOLD MSE: ",mean(scores))
    lin_error_arr.append(mean(scores))
    if abs(0 - mean(scores)) < min_error:
        min_error = mean(scores)
        best_fold = i
    
print()
print("BEST FOLD = ", best_fold)
print("MSE = ", min_error)

lin_error_data = [[lin_value[i],lin_error_arr[i]] for i in range(len(lin_value))]
lin_error_df = pd.DataFrame(data = lin_error_data, columns = ["FOLD NUM", "MEAN ERROR"])

2 -FOLD MSE:  -3.156363177433732e-06
3 -FOLD MSE:  -3.1682521592808067e-06
4 -FOLD MSE:  -3.1573860584874485e-06
5 -FOLD MSE:  -3.168721015165281e-06
6 -FOLD MSE:  -3.164398560785279e-06
7 -FOLD MSE:  -3.1637098964556984e-06
8 -FOLD MSE:  -3.165100618828823e-06
9 -FOLD MSE:  -3.1683009677700534e-06
10 -FOLD MSE:  -3.1668664372856333e-06
11 -FOLD MSE:  -3.165946066900104e-06
12 -FOLD MSE:  -3.1675857489747235e-06
13 -FOLD MSE:  -3.1686609174881373e-06
14 -FOLD MSE:  -3.164797702399943e-06
15 -FOLD MSE:  -3.170369310927235e-06
16 -FOLD MSE:  -3.1695366691954734e-06
17 -FOLD MSE:  -3.1679607317946308e-06
18 -FOLD MSE:  -3.1680245800428067e-06
19 -FOLD MSE:  -3.1678593601502723e-06
20 -FOLD MSE:  -3.1676071612678327e-06
21 -FOLD MSE:  -3.1676488734496793e-06
22 -FOLD MSE:  -3.1720000057370163e-06
23 -FOLD MSE:  -3.171670327051389e-06
24 -FOLD MSE:  -3.172303994436492e-06
25 -FOLD MSE:  -3.170098178473087e-06
26 -FOLD MSE:  -3.1696663874644003e-06
27 -FOLD MSE:  -3.16888420782877e-06
28 -FO

In [21]:
linreg.fit(X_train,y_train)
predicted = linreg.predict(X_test)


mse = mean_squared_error(predicted,y_test)

dat = [[predicted[i],y_test[i]] for i in range(len(y_test)) ]
squared_error = (y_test - predicted)**2

print("MSE: ",mse)
    

#create dataframe
lin_df = pd.DataFrame(data = dat,columns = ["Predicted USD","Actual USD"])
lin_df["Squared Error"] = squared_error

2.7517168641448486e-06


In [22]:
#PART A
#Lasso regression
lasso = Lasso()
best_fold = 2
min_error = 30
lasso_error_arr = []
lasso_value = [i for i in range(2,30)] 
for i in range(2,30):
    cv = KFold(n_splits=i, random_state=1, shuffle=True)
    scores = cross_val_score(lasso, X_train, y_train,scoring='neg_mean_squared_error',cv=cv)
    print(i,"-FOLD MSE: ",mean(scores))
    lasso_error_arr.append(mean(scores))
    if abs(0 - mean(scores)) < min_error:
        min_error = mean(scores)
        best_fold = i
    
print()
print("BEST FOLD = ", best_fold)
print("MSE = ", min_error)

lasso_error_data = [[lasso_value[i],lasso_error_arr[i]] for i in range(len(lasso_value))]
lasso_error_df = pd.DataFrame(data = lasso_error_data, columns = ["FOLD NUM", "MEAN ERROR"])

2 -FOLD MSE:  -6.810237794961967e-06
3 -FOLD MSE:  -6.818358052479679e-06
4 -FOLD MSE:  -6.8090003000834516e-06
5 -FOLD MSE:  -6.825051967524818e-06
6 -FOLD MSE:  -6.821673711782167e-06
7 -FOLD MSE:  -6.814143037366277e-06
8 -FOLD MSE:  -6.820385918624452e-06
9 -FOLD MSE:  -6.814232294380453e-06
10 -FOLD MSE:  -6.817369480078784e-06
11 -FOLD MSE:  -6.816190184166904e-06
12 -FOLD MSE:  -6.818214482725792e-06
13 -FOLD MSE:  -6.817717101806424e-06
14 -FOLD MSE:  -6.8177102078360065e-06
15 -FOLD MSE:  -6.815832864083116e-06
16 -FOLD MSE:  -6.818926813901846e-06
17 -FOLD MSE:  -6.814557009371047e-06
18 -FOLD MSE:  -6.818999036119136e-06
19 -FOLD MSE:  -6.819300256608799e-06
20 -FOLD MSE:  -6.816129435878548e-06
21 -FOLD MSE:  -6.81774586701582e-06
22 -FOLD MSE:  -6.814387124358032e-06
23 -FOLD MSE:  -6.817170019573228e-06
24 -FOLD MSE:  -6.817843601893272e-06
25 -FOLD MSE:  -6.818288653392212e-06
26 -FOLD MSE:  -6.817406405628474e-06
27 -FOLD MSE:  -6.817379744721769e-06
28 -FOLD MSE:  -6.8

In [23]:
lasso.fit(X_train,y_train)
predicted = lasso.predict(X_test)



In [24]:
dat = [[predicted[i],y_test[i]] for i in range(len(y_test)) ]
lass_df = pd.DataFrame(data = dat,columns = ["Predicted USD","Actual USD"])

squared_error = (y_test - predicted)**2
lass_df["Squared Error"] = squared_error

print(mean(squared_error))

6.797314216984891e-06


In [13]:
#OUTPUT TO CSV FILES
lass_df.to_csv("./lass_df.csv")
lin_df.to_csv("./lin_df.csv")
df.to_csv("./training_data.csv")
lin_error_df.to_csv("./lin_error.csv")
lasso_error_df.to_csv("./lasso_error.csv")

In [14]:
lass_df

,Predicted USD,Actual USD,Squared Error
0,0.02669,0.026000,4.758974e-07
1,0.02669,0.030937,1.803493e-05
2,0.02669,0.028986,5.268778e-06
3,0.02669,0.025655,1.071994e-06
4,0.02669,0.025989,4.914149e-07
...,...,...,...
406,0.02669,0.025572,1.251218e-06
407,0.02669,0.025161,2.337395e-06
408,0.02669,0.025557,1.283622e-06
409,0.02669,0.027225,2.856877e-07


In [15]:
lin_df

,Predicted USD,Actual USD,Squared Error
0,0.025169,0.026000,6.908792e-07
1,0.028985,0.030937,3.808878e-06
2,0.024999,0.028986,1.589460e-05
3,0.024382,0.025655,1.621106e-06
4,0.025828,0.025989,2.597642e-08
...,...,...,...
406,0.022810,0.025572,7.627191e-06
407,0.024855,0.025161,9.359294e-08
408,0.026473,0.025557,8.381700e-07
409,0.028447,0.027225,1.492784e-06


In [16]:
lin_error_df

,FOLD NUM,MEAN ERROR
0,2,-0.000003
1,3,-0.000003
2,4,-0.000003
3,5,-0.000003
4,6,-0.000003
5,7,-0.000003
6,8,-0.000003
7,9,-0.000003
8,10,-0.000003
9,11,-0.000003


In [17]:
lasso_error_df

,FOLD NUM,MEAN ERROR
0,2,-0.000007
1,3,-0.000007
2,4,-0.000007
3,5,-0.000007
4,6,-0.000007
5,7,-0.000007
6,8,-0.000007
7,9,-0.000007
8,10,-0.000007
9,11,-0.000007
